In [1]:
# Third test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
# Global Setup
# Creating temp local repos to test on
TMPA=/tmp/temp-git-test

Cloning into '/tmp/temp-git-test'...
remote: Enumerating objects: 488, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 488 (delta 49), reused 83 (delta 49), pack-reused 405
Receiving objects: 100% (488/488), 42.34 KiB | 535.00 KiB/s, done.
Resolving deltas: 100% (286/286), done.
From https://github.com/LorenzoFerraces/bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
[main e344142] first commit on second local
 1 file changed, 1 insertion(+)
 create mode 100644 hello.txt
Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 4 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 310 bytes | 310.00 KiB/s, done.
Total 3 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   9f44c53..e344142  main -> main


In [ ]:
# Global setup
# Define aliases for encoding output (e.g., to ensure output the same across runs).
## OLD: alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/\n/<nl>/g;  s/[\x00-\x31]/<cc>/g;"'
alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/[\x00-\x08\x11\x31]/<cc>/g;"'
alias encode-nums='perl -pe "s/[0-9]+/N/g;"'
## TODO: define a better hex encoder to avoid false positives (e.g., abc).
alias encode-hex='perl -pe "s/[0-9A-F]{4,}/<hex>/ig;"'
# note: uses perl slurp mode (i.e., read entire file) to check for consecutive newlines
alias encode-newlines='perl -0777 -pe "while (/\n\n/) { s/\n\n/\n<newline>\n/g; }"'
function encode-text () { encode-control-chars | encode-nums | encode-hex | encode-newlines; }

In [ ]:
# Global Setup
git clone https://github.com/LorenzoFerraces/bash-test.git "$TMPA" > /dev/null 2>&1

In [ ]:
# Global Setup
# Move to second local to push some changes
command cd "$TMPA" > /dev/null 2>&1
git pull origin main > /dev/null 2>&1
echo "hello second local" > hello.txt 
git add hello.txt > /dev/null 2>&1
git commit -m "first commit on second local" > /dev/null 2>&1
git push > /dev/null 2>&1

In [3]:
# git-diff-plus tests
cat hello.txt > bye.txt
echo "differ" >> hello.txt
diffs=$(git-diff-plus hello.txt bye.txt | grep "^\+\w" | wc -l)

In [4]:
# Assert 
# there should only be one line of difference
[ $diffs -eq 1 ]; echo $?;

0


In [5]:
# git-diff-list tests
changed=$(git-diff-list | wc -l)

In [6]:
# Assert 
# there should be one file changed
[ $changed -eq 1 ]; echo $?;

0


In [7]:
# git-vdiff-alias tests
killall nvim > /dev/null 2>&1
git-vdiff-alias | encode-text
instances=$(ps -e | grep "nvim" | wc -l)

nvim: no process found
[1] 15207

[1]+  Stopped                 git-difftool-plus "$@"


In [8]:
# Assert there is an instance of nvim 
[ $instances -eq 1 ]; echo $?;

0


In [9]:
# git-next-checkin tests
rm -f bye.txt > /dev/null 2>&1
previous_changes=$( git status -s | wc -l )

removed 'bye.txt'


In [ ]:
# set env variables and run git-next-checkin
export GIT_MESSAGE="checkin"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
git-next-checkin hello.txt | encode-text

fatal: Unable to read current working directory: No such file or directory
[2]-  Exit 128                ( sleep $delay; git-difftool-plus "$mod_file" )  (wd: /tmp/temp-git-test)
(wd now: /tmp)


In [ ]:
# delete new files created by git-next-cheking and count changes
rm -rf $(ls | grep '[.][0-9]*[.]') > /dev/null 2>&1
changes=$( git status -s | wc -l )

In [11]:
# Assert there were changes before, and no changes now
[ $previous_changes -gt 1 ] && [ $changes -eq 0 ]; echo $?;

0
0


In [12]:
# undo changes and remove newly created files
# TODO2:
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null 2>&1
rm -rf $(ls | grep '[.][0-9]*[.]') > /dev/null 2>&1
git rm -f *.txt > /dev/null 2>&1
git add . > /dev/null 2>&1

[main dca425a] cleanup test files
 1 file changed, 2 deletions(-)
 delete mode 100644 hello.txt
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 4 threads
Compressing objects: 100% (1/1), done.
Writing objects: 100% (2/2), 245 bytes | 245.00 KiB/s, done.
Total 2 (delta 0), reused 1 (delta 0), pack-reused 0
To https://github.com/LorenzoFerraces/bash-test.git
   fa905c1..dca425a  main -> main


In [ ]:
## Lorenzo-TODO2: use one cell per output (to make this less brittle)
git commit -m "cleanup test files" > /dev/null 2>&1
git push > /dev/null 2>&1

In [13]:
## OLD: extraneous command as full path can be used as below
## # delete test local repos
## command cd "/tmp"

In [ ]:
## OLD:
## rm -rf temp-git-test > /dev/null
## Lorenzo-Note: useful for retaining directories for post-hoc analysis
rename-with-file-date /tmp/temp-git-test > /dev/null 2>&1
export GIT_NO_CONFIRM=$previous